In [2]:
!pip install --upgrade pip setuptools wheel -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.8 MB/s eta 0:00:00


In [3]:
!pip install requests semanticscholar crossrefapi PyMuPDF -q


In [4]:
import json
import os
from datetime import datetime
import requests
from crossref.restful import Works
from semanticscholar import SemanticScholar
import fitz  # PyMuPDF


🔍 1. Search Semantic Scholar

In [5]:
def search_semantic_scholar(query, max_results=10):
    """
    Searches Semantic Scholar for research papers
    based on a topic input.
    Returns structured metadata for each paper.
    """
    print("🔍 Searching Semantic Scholar...")
    sch = SemanticScholar(timeout=10)

    papers = []
    try:
        # Perform API search
        results = sch.search_paper(query, limit=max_results)

        # Extract fields from each paper
        for p in results:
            papers.append({
                "title": p.title,
                "authors": [a.name for a in p.authors] if p.authors else [],
                "year": p.year,
                "pdf_url": p.openAccessPdf.get("url") if p.openAccessPdf else None,
                "citationCount": p.citationCount,
                "abstract": p.abstract,
                "url": p.url
            })
    except Exception as e:
        print("Semantic Scholar error:", e)

    return papers


In [6]:
def search_semantic_scholar(query, max_results=10):
    """
    Searches Semantic Scholar for research papers
    based on a topic input.
    Returns structured metadata for each paper.
    """
    print("🔍 Searching Semantic Scholar...")
    sch = SemanticScholar(timeout=10)

    papers = []
    try:
        # Perform API search
        results = sch.search_paper(query, limit=max_results)

        # Extract fields from each paper
        for p in results:
            papers.append({
                "title": p.title,
                "authors": [a.name for a in p.authors] if p.authors else [],
                "year": p.year,
                "pdf_url": p.openAccessPdf.get("url") if p.openAccessPdf else None,
                "citationCount": p.citationCount,
                "abstract": p.abstract,
                "url": p.url
            })
    except Exception as e:
        print("Semantic Scholar error:", e)

    return papers


📚 2. Search CrossRef (Backup API)

In [7]:
def search_crossref(query, max_results=5):
    """
    Searches CrossRef as a fallback API source.
    Useful when Semantic Scholar returns fewer results.
    """
    print("🔍 Searching CrossRef...")
    works = Works()
    papers = []

    try:
        # Search CrossRef and sort by relevance (score)
        results = works.query(query).sort("score")

        count = 0
        for item in results:
            if count >= max_results:
                break

            # Extract minimal metadata
            papers.append({
                "title": item.get("title", [""])[0],
                "authors": [a.get("family","") for a in item.get("author", [])],
                "year": item.get("issued", {}).get("date-parts", [[None]])[0][0],
                "pdf_url": None,            # CrossRef does not give PDFs
                "citationCount": None,
                "abstract": None,
                "url": item.get("URL", "")
            })
            count += 1

    except Exception as e:
        print("CrossRef error:", e)

    return papers


🧠 3. Main Function — Combine Both APIs & Save JSON

In [8]:
def run_module1(topic):
    """
    Runs the full Module-1 workflow:
    1. Search Semantic Scholar
    2. Search CrossRef
    3. Combine results
    4. Save output to JSON
    """
    print("\n===============================")
    print("MODULE 1: Paper Search Started")
    print("===============================\n")

    # API calls
    ss_results = search_semantic_scholar(topic, max_results=10)
    cr_results = search_crossref(topic, max_results=5)

    # Merge results
    all_papers = ss_results + cr_results

    print(f"\n📚 Total papers found: {len(all_papers)}")

    # Create output folder
    os.makedirs("data/search_results", exist_ok=True)

    # Dynamic filename
    filename = f"data/search_results/search_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

    # Save JSON data
    with open(filename, "w", encoding="utf-8") as f:
        json.dump({
            "topic": topic,
            "papers": all_papers
        }, f, indent=4, ensure_ascii=False)

    print("\n✅ Module 1 Completed!")
    print(f"📁 Results saved to: {filename}")

    return filename


▶ Run Module-1

In [9]:
topic = "AI system to automatically review and summarize research papers"
run_module1(topic)



MODULE 1: Paper Search Started

🔍 Searching Semantic Scholar...
🔍 Searching CrossRef...

📚 Total papers found: 16

✅ Module 1 Completed!
📁 Results saved to: data/search_results/search_results_20251211_195444.json


'data/search_results/search_results_20251211_195444.json'

🧩 MODULE 2 — PDF Filtering, Ranking, Downloading
Select PDFs → Download with Verification → Save Report

📥 1. Load Search Results

In [10]:
def load_search_results(filepath=None):
    """
    Loads the JSON file generated in Module-1.
    """
    if not filepath:
        print("❌ ERROR: Provide a valid filepath.")
        return None

    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f" Loaded {len(data['papers'])} papers on topic: {data['topic']}")
        return data
    except Exception as e:
        print(f" Error loading search results: {e}")
        return None


📝 2. Filter Papers Having PDF Links

In [11]:
def filter_papers_with_pdfs(papers):
    """
    Selects only papers that have a valid PDF URL.
    """
    papers_with_pdf = [
        p for p in papers
        if p.get("pdf_url") and ("pdf" in p["pdf_url"].lower())
    ]

    print(f"\n PDF Check:")
    print(f" • Total papers: {len(papers)}")
    print(f" • Papers with PDF URLs: {len(papers_with_pdf)}")

    return papers_with_pdf


⭐ 3. Rank Papers (Citations + Year)

In [12]:
def rank_papers(papers):
    """
    Sorts papers by highest citation count and recent year.
    """
    valid = [
        p for p in papers
        if p.get("citationCount") is not None and p.get("year")
    ]

    ranked = sorted(valid, key=lambda x: (x["citationCount"], x["year"]), reverse=True)
    return ranked


🎯 4. Select Top N Papers

In [13]:
def select_top_papers(papers, count=3):
    """
    Selects the top N papers after:
    1. PDF filtering
    2. Ranking
    """
    papers_with_pdf = filter_papers_with_pdfs(papers)
    ranked = rank_papers(papers_with_pdf)
    selected = ranked[:count]

    print(f"\n Top {count} Selected Papers:")
    for i, p in enumerate(selected):
        print(f"\n{i+1}. {p['title']}")
        print(f"   ↳ Citations: {p['citationCount']}, Year: {p['year']}")

    return selected


 5. PDF Verification

In [14]:
def verify_pdf(filepath):
    """
    Checks if the file is a valid PDF using PyMuPDF.
    Size <1KB is marked invalid.
    """
    try:
        if not os.path.exists(filepath):
            return False
        if os.path.getsize(filepath) < 1024:
            return False
        with fitz.open(filepath) as doc:
            return len(doc) > 0
    except:
        return False


⬇ 6. Download PDF with Retry + Verification

In [15]:
def download_pdf_with_verification(url, filename, max_retries=3):
    """
    Downloads a PDF from a URL.
    Includes:
    - Retry logic
    - User-Agent header
    - PDF file validation
    """
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Accept': 'application/pdf'
    }

    for attempt in range(max_retries):
        try:
            print(f"  Attempt {attempt + 1}/{max_retries}...")
            response = requests.get(url, headers=headers, timeout=30, stream=True)

            # Blocked access
            if response.status_code == 403:
                print("    HTTP 403 Forbidden. Retrying...")
                continue

            if response.status_code != 200:
                print(f"    HTTP Error: {response.status_code}")
                continue

            # Save PDF
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

            # Validate
            if verify_pdf(filename):
                print(f"    ✅ Downloaded: {os.path.getsize(filename):,} bytes")
                return True
            else:
                print("    ❌ Invalid PDF. Retrying...")
                os.remove(filename)

        except Exception as e:
            print(f"    Error: {str(e)[:50]}")

    return False


📩 7. Download All Selected Papers

In [16]:
def download_selected_papers(selected, output_dir="downloads"):
    """
    Downloads the top N research paper PDFs into a folder.
    """
    os.makedirs(output_dir, exist_ok=True)
    downloaded = []

    for i, paper in enumerate(selected):
        print(f"\n[{i+1}/{len(selected)}] Downloading:")
        print(" ", paper['title'])

        # Safe filename handling
        safe_title = "".join(c for c in paper['title'] if c.isalnum())[:30]
        filename = f"{output_dir}/{safe_title}.pdf"

        if download_pdf_with_verification(paper["pdf_url"], filename):
            print("   ✅ Success:", filename)
            paper["local_file"] = filename
            downloaded.append(paper)
        else:
            print("   ❌ Failed")

    return downloaded


📄 8. Save JSON Download Report

In [17]:
def save_download_report(downloaded, topic):
    """
    Creates a summary report of downloaded papers.
    """
    os.makedirs("data/reports", exist_ok=True)

    report = {
        "topic": topic,
        "download_count": len(downloaded),
        "papers": downloaded
    }

    output_file = f"data/reports/download_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=4)

    print("\n Report saved to:", output_file)
    return output_file


🚀 9. Final Master Function (Module-2)

In [22]:
# =======================================================
# AUTO-LOAD LATEST SEARCH RESULTS
# =======================================================
import os
import glob

def get_latest_search_results():
    """
    Finds the most recent JSON file from data/search_results/
    """
    folder_path = "data/search_results/"
    json_files = glob.glob(os.path.join(folder_path, "*.json"))

    if not json_files:
        raise FileNotFoundError("❌ No search results found. Run Module 1 first.")

    latest_file = max(json_files, key=os.path.getctime)
    print(f"📄 Latest search results auto-loaded:\n{latest_file}")
    return latest_file


In [23]:
def main_module2(top_n=3):
    """
    MODULE 2 DRIVER FUNCTION
    ------------------------
    Auto-loads latest search results, selects top papers,
    downloads PDFs, and generates a report.
    """

    print("\n========== MODULE 2 STARTED ==========\n")

    # Step 1: Auto-load latest search_results file
    try:
        json_path = get_latest_search_results()
    except Exception as e:
        print(f"❌ ERROR: Failed to load latest search results.\n{e}")
        return None

    # Step 2: Load JSON content
    data = load_search_results(json_path)
    if not data:
        print("❌ ERROR: search_results JSON is empty or unreadable.")
        return None

    # Step 3: Filter & rank papers
    print("\n📊 Selecting top papers...\n")
    selected = select_top_papers(data["papers"], count=top_n)

    # Step 4: Download selected PDFs
    print("\n📥 Downloading PDFs...\n")
    downloaded = download_selected_papers(selected)

    # Step 5: Save results
    save_download_report(downloaded, data["topic"])

    print("\n========== MODULE 2 COMPLETED ==========\n")

    return downloaded


In [24]:
main_module2(top_n=3)



========== MODULE 2 STARTED ==========

📄 Latest search results auto-loaded:
data/search_results/search_results_20251211_195444.json
 Loaded 16 papers on topic: AI system to automatically review and summarize research papers

📊 Selecting top papers...


 PDF Check:
 • Total papers: 16
 • Papers with PDF URLs: 4

 Top 3 Selected Papers:

1. Automatic assessment of text-based responses in post-secondary education: A systematic review
   ↳ Citations: 91, Year: 2023

2. Editorial for Special Issue on Large-scale Pre-training: Data, Models, and Fine-tuning
   ↳ Citations: 2, Year: 2023

3. Special issue on future hybrid artificial intelligence and machine learning for smart expert systems
   ↳ Citations: 0, Year: 2021

📥 Downloading PDFs...


[1/3] Downloading:
  Automatic assessment of text-based responses in post-secondary education: A systematic review
  Attempt 1/3...
    ✅ Downloaded: 962,703 bytes
   ✅ Success: downloads/Automaticassessmentoftextbased.pdf

[2/3] Downloading:
  Editor

[{'title': 'Automatic assessment of text-based responses in post-secondary education: A systematic review',
  'authors': ['Rujun Gao',
   'H. Merzdorf',
   'S. Anwar',
   'M. C. Hipwell',
   'Arun Srinivasa'],
  'year': 2023,
  'pdf_url': 'https://arxiv.org/pdf/2308.16151',
  'citationCount': 91,
  'abstract': 'Text-based open-ended questions in academic formative and summative assessments help students become deep learners and prepare them to understand concepts for a subsequent conceptual assessment. However, grading text-based questions, especially in large courses, is tedious and time-consuming for instructors. Text processing models continue progressing with the rapid development of Artificial Intelligence (AI) tools and Natural Language Processing (NLP) algorithms. Especially after breakthroughs in Large Language Models (LLM), there is immense potential to automate rapid assessment and feedback of text-based responses in education. This systematic review adopts a scientific and r